<div style="background: linear-gradient(45deg, #aaa, #eee, #fff, #eee, #fff, #eee, #aaa);
  text-align: center;
  padding-top: 25px;
            padding-bottom: 25px;"><h1 style ="font-size: 30px;">Lab 4</h1><h2>✨ More Sequential Circuits ✨</h2></div>
<div style="font-size: 15px;">

### Import the necessary Chisel dependencies. 

In [ ]:
interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

## Problem 1 - Ticker
In the previous lab and in the lecture we saw multiple designs of counters, i.e., modules that count the number of clock ticks up to a particular value. Here we want to implement a so called `Ticker` that outputs $1$ whenever the number of clock cycles is divisible by $n$. For example for $n=2$ it should output $1$ in every other clock cycle.

To build such a `Ticker` we want to reuse our Counter from the lecture, which is reproduced below:

In [ ]:
class Counter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    when (io.en) {
        when (count < maxVal.U) {
            count := count + 1.U
        } .otherwise {
            count := 0.U
        }
    }
    io.out := count
}

Construct a `Ticker` using the above Counter module.

In [ ]:
class Ticker(nTicks: Int) extends Module {
    ???
}

Now it is time to test your `Ticker`, the skeleton is given and works as follows, we want to check the correctness of output for $3$ rounds. Here one round corresponds to $1$ tick of the `Ticker`.

In [ ]:
def testTicker(nTicks: Int): Boolean = {
    test(new Ticker(nTicks)) { dut =>
        for (round <- 0 until 3) {
            ???
        }
    }
    println("Success!")
    true
}

In [ ]:
assert(testTicker(1))
assert(testTicker(2))
assert(testTicker(3))
assert(testTicker(4))

## Problem 2 - Accumulator
Now, let's build an `Accumulator`. Each cycle `en` is high, it should add `in` to it's internal total. The internal total is visible as the output `out`. The internal total should initialize to $0$ on reset. The reset happens automatically whenever the implicit `reset` input is true on the circuit and can be triggered by `dut.reset.poke(1.B)`. See the test function below to better understand this.

In [ ]:
class Accumulator(w: Int) extends Module {
    ???
}

In [ ]:
class Accumulator(w: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(w.W))
        val en  = Input(Bool())
        val out = Output(UInt(w.W))
    })
    val total = RegInit(0.U(w.W))
    when(io.en) {
        total := total + io.in
    }
    io.out := total
}

This time the test is already provided by us, but make sure to read through it and understand what happens in each cycle, especially in the last one.

In [ ]:
def testAccumulator: Boolean = {
    test(new Accumulator(4)) { dut =>
        // Cycle 0
        dut.io.in.poke(0.U)
        dut.io.en.poke(0.B)
        dut.clock.step(1)
        dut.io.out.expect(0.U)

        // Cycle 1
        dut.io.in.poke(0.U)
        dut.io.en.poke(0.B)
        dut.clock.step(1)
        dut.io.out.expect(0.U)

        // Cycle 2
        dut.io.in.poke(3.U)
        dut.io.en.poke(0.B)
        dut.clock.step(1)
        dut.io.out.expect(0.U)

        // Cycle 3
        dut.io.in.poke(3.U)
        dut.io.en.poke(1.B)
        dut.clock.step(1)
        dut.io.out.expect(3.U)

        // Cycle 4
        dut.io.in.poke(4.U)
        dut.io.en.poke(1.B)
        dut.clock.step(1)
        dut.io.out.expect(7.U)        

        //Cycle 5
        dut.io.in.poke(1.U)
        dut.io.en.poke(1.B)
        dut.reset.poke(1.B) // when reset is true, all registers are reset to their default values
        dut.clock.step(1)
        dut.io.out.expect(0.U)
    }
    println("Success!")
    true
}
assert(testAccumulator)

## Problem 3 - Histogram
Let's build a `Histogram` generator. The generated hardware shall count how often it sees each input. It will take a parameter `n`, and internally it will use `n` registers to count how often it has seen inputs (from 0 to n-1). Each cycle, the `x` input chooses which internal register to increment. The `out` output should provide the new total value that will be written into the register the next cycle. In other words, `out` immediately shows the new value of the count associated with `x`.

Implement the `Histogram` generator using Chisel registers and vectors.

In [ ]:
class Histogram(n: Int) extends Module {
    ???
}

In [ ]:
class Histogram(n: Int) extends Module {
    val io = IO(new Bundle {
        val x = Input(UInt(log2Ceil(n).W))
        val out = Output(UInt(5.W))
    })
    
    val regvec = RegInit(VecInit(Seq.fill(n)(0.U(5.W))))
    
    regvec(io.x) := regvec(io.x) + 1.U
    io.out := regvec(io.x)
}

Here is a test for the `Histogram` generator:

In [ ]:
def testHistogram: Boolean = {
    test(new Histogram(3)) { dut =>    
        // Cycle 0
        dut.io.x.poke(1.U)
        dut.clock.step(1)
        dut.io.out.expect(1.U)

        // Cycle 1
        dut.io.x.poke(1.U)
        dut.clock.step(1)
        dut.io.out.expect(2.U)
        
        // Cycle 2
        dut.io.x.poke(0.U)
        dut.clock.step(1)
        dut.io.out.expect(1.U)

        // Cycle 3
        dut.io.x.poke(2.U)
        dut.clock.step(1)
        dut.io.out.expect(1.U)
    }
    println("Success!")
    true
}

In [ ]:
assert(testHistogram)